In [3]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain.docstore.document import Document
from nltk.corpus import wordnet

# Ensure NLTK resources are downloaded
import nltk
nltk.download('wordnet')

# Authenticate with the Kaggle API
api = KaggleApi()
api.authenticate()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\neals\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Download the "All The News" dataset from Kaggle
api.dataset_download_files('asad1m9a9h6mood/news-articles', path='data/', unzip=True)

# Load and process the dataset
file_path = 'data/Articles.csv'  # Adjust the path based on dataset name
df = pd.read_csv(file_path, encoding='ISO-8859-1')


Dataset URL: https://www.kaggle.com/datasets/asad1m9a9h6mood/news-articles


In [5]:
# Convert documents into the required format for Chroma
document_list = [
    Document(
        page_content=row['Article'],
        metadata={
            'date': row['Date'],
            'heading': row['Heading'],
            'news_type': row['NewsType']
        }
    )
    for _, row in df.iterrows()
]

# TODO Truncate document list because it takes too long to process
if len(document_list) > 1000:
    document_list = document_list[:1000]


In [6]:
# Embed documents using SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# Create a vector store using Chroma
vector_store = Chroma.from_documents(document_list, embeddings)


c:\Users\neals\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\neals\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [7]:
# Define a basic query 2.1
query = "Tell me about the latest news in the world and include the date"

# Perform semantic search and retrieve relevant documents
retrieved_docs = vector_store.similarity_search(query, k=5)

# Step 5: Generate a Response Using Retrieved Documents as Context
context = " ".join([doc.page_content[:1000] for doc in retrieved_docs[:2]])  # Limit context to 2 docs and 1000 chars



: 

In [8]:
# Get the Hugging Face token from the environment variables
hf_token = os.getenv("HUGGING_FACE_TOKEN")
#print(hf_token)

# Use an LLM for response generation, in this case GPT2
#llm = pipeline("text-generation", model="gpt2")  # Replace with another model if needed
#for VM usage
#llm = pipeline("text-generation", model="/home/nealsharma/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/7231864981174d9bee8c7687c24c8344414eae6b")

llm = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1", token=hf_token)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Start of 2.2
# Function to expand the query using WordNet

def expand_query_with_synonyms(query):
    synonyms = set()
    for word in query.split():
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
    expanded_query = query + " " + " ".join(synonyms)
    return expanded_query


In [ ]:
# Expand the initial query
expanded_query = expand_query_with_synonyms("Tell me about the latest news in AI")

# Perform semantic search and retrieve relevant documents
retrieved_docs = vector_store.similarity_search(expanded_query, k=5)



In [ ]:
# Combine top-ranked documents into a context string
context = " ".join([doc.page_content for doc in retrieved_docs[:2]])  # Limiting to top 2 documents

# Get the Hugging Face token from the environment variables
hf_token = os.getenv("HUGGING_FACE_TOKEN")


# Initialize the LLM (using Mistral or GPT-2 as an example)
llm = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1", token=hf_token)

# Generate the final response
response = llm(context, max_new_tokens=100, do_sample=True, top_k=50)[0]['generated_text']
print(response)


Loading checkpoint shards: 100%|██████████| 2/2 [00:30<00:00, 15.05s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 